In [20]:
import os
cwd = os.getcwd()
pygad_loc = '/grid/home/nbourgeois/codonOpt'
os.chdir(pygad_loc)
from general_functions import *
from metrics import *
os.chdir(cwd)

In [21]:
ga_input = '/grid/home/nbourgeois/data/codon_jason/final_seq.fa'
output = '' #output filename
tissue = 'Liver'

In [22]:
def get_codon_weights(loc):
    # read table
    try:
        index_loc = os.path.join(pygad_loc,loc)
        codon_table_raw = pd.read_csv(index_loc,sep='\t')

    except FileNotFoundError:
        print('Invalid Tissue')
        return()    

    
    try:
        codon_table = codon_table_raw.set_index('#Codon')
    except KeyError:
        codon_table = codon_table_raw.set_index('Codon')
    maxes = codon_table_raw.groupby(by=['AA'])['Frequency'].max()
    codon_table['max_col'] = codon_table.apply(lambda x: maxes[x['AA']],axis=1)
    codon_table['weight'] = codon_table['Frequency'] / codon_table['max_col']
    weight_dict = codon_table['weight'].to_dict()
    
    return(weight_dict)

def get_bicodon_weights(loc):
    
    def bicodon_to_AA(string):
        cod1 = string[:3]
        cod2 = string[3:]
        biAA = forward_table[cod1] + forward_table[cod2]
        return(biAA)
    
    def get_forward_table():
        codon_usage_table_loc = os.path.join(pygad_loc,'references/codon_usage.getex.txt')
        codon_usage_table = pd.read_csv(codon_usage_table_loc,sep='\t')
        forward_table = pd.Series(codon_usage_table.AA.values,index=codon_usage_table.Codon).to_dict()
        return(forward_table)
     
    # read table
    try:
        index_loc = os.path.join(pygad_loc,loc)
        codon_table_raw_t = pd.read_csv(index_loc,sep='\t')

    except FileNotFoundError:
        print('Invalid Tissue')
        return()    
    
    codon_table = codon_table_raw_t.T

    # add amino acid pair column
    bicodons_nt = list(codon_table.index)
    forward_table = get_forward_table()
    codon_table['AApair'] = list(map(bicodon_to_AA,bicodons_nt))
    total = codon_table[0].sum()
    
    #add frequency
    bicodons_nt = list(codon_table.index)
    codon_table['AApair'] = list(map(bicodon_to_AA,bicodons_nt))
    total = codon_table[0].sum()
    codon_table['Frequency'] = codon_table.apply(lambda x: (x[0] / total * 1000) ,axis=1)
    
    #calculate weights
    maxes = codon_table.groupby(by=['AApair'])['Frequency'].max()
    codon_table['max_col'] = codon_table.apply(lambda x: maxes[x['AApair']],axis=1)
    codon_table['weight'] = codon_table['Frequency'] / codon_table['max_col']
    
    weight_dict = codon_table['weight'].to_dict()
    
    return(weight_dict)

In [23]:
(fastaIDs, fastaSeqs) = readFasta(ga_input)
names = ['_'.join(anid.split(':')[1].split('_')[0:2]) for anid in fastaIDs]

# CAI

In [24]:

tissue = 'Liver'
orig_weights = get_codon_weights('references/CoCoPUTs_codon_usage/codon_usage/{}.codon.txt'.format(tissue))
remade_weights = get_codon_weights('tissue_cocoputs/data/codon_tables/GTEx_{}_codon_orig_v7.tsv'.format(tissue))
hi_weights = get_codon_weights('tissue_cocoputs/data/codon_tables/GTEx_{}_codon_hifr_v7.tsv'.format(tissue))

orig_cais = []
new_cais = []
hi_cais = []
diffs = []
hidiffs = []
for fid,aa_seq in zip(fastaIDs, fastaSeqs):
    orig_cai = get_cai(aa_seq, orig_weights) *100
    orig_cais.append(orig_cai)
    new_cai = get_cai(aa_seq, remade_weights)*100
    new_cais.append(new_cai)
    hi_cai = get_cai(aa_seq, hi_weights)*100
    hi_cais.append(hi_cai)
    diffs.append(new_cai - orig_cai)
    hidiffs.append(hi_cai - orig_cai)
cai_table = pd.DataFrame(data = {'Name':names,'Orig':orig_cais,'New':new_cais,'Diff':diffs,'High':hi_cais,'Hi-Diff':hidiffs})
cai_table['Diff'].max()

0.5371632959562902

# BAI

In [29]:
tissue = 'Lung'
orig_weights = get_bicodon_weights('references/CoCoPUTs_codon_usage/bicodon_usage/Liver.bicodon.txt')
remade_weights = get_bicodon_weights('tissue_cocoputs/data/bicodon_tables/GTEx_{}_bicodon_orig_v7.tsv'.format(tissue))
hi_weights = get_bicodon_weights('tissue_cocoputs/data/bicodon_tables/GTEx_{}_bicodon_test_v7.tsv'.format(tissue))

/data/software/anaconda3/envs/bfx-codon/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in double_scalars


In [30]:
orig_bai = get_bai(aa_seq, orig_weights) *100

hi_bai = get_bai(aa_seq, hi_weights)*100


In [31]:

orig_bais = []
new_bais = []
hi_bais = []
diffs = []
hidiffs = []
for fid,aa_seq in zip(fastaIDs, fastaSeqs):
    orig_bai = get_bai(aa_seq, orig_weights) *100
    orig_bais.append(orig_bai)
    new_bai = get_bai(aa_seq, remade_weights)*100
    new_bais.append(new_bai)
    hi_bai = get_bai(aa_seq, hi_weights)*100
    hi_bais.append(hi_bai)
    diffs.append(new_bai - orig_bai)
    hidiffs.append(hi_bai - orig_bai)
bai_table = pd.DataFrame(data = {'Name':names,'Orig':orig_bais,'New':new_bais,'Diff':diffs,'High':hi_bais,'Hi-Diff':hidiffs})
bai_table['Diff'].max()

6.1497085596250685

In [32]:
bai_table

,Name,Orig,New,Diff,High,Hi-Diff
0,cai_0,49.636312,53.695463,4.059151,NaN,NaN
1,cai_1,55.389658,59.018071,3.628413,NaN,NaN
2,cai_2,56.645103,60.702536,4.057433,NaN,NaN
3,cpg_0,35.003562,40.899825,5.896263,NaN,NaN
4,cpg_1,34.202749,39.929349,5.726599,NaN,NaN
5,cpg_2,37.656560,43.806269,6.149709,NaN,NaN
6,bai_0,71.307436,71.686982,0.379546,NaN,NaN
7,bai_1,71.161317,72.233356,1.072039,NaN,NaN
8,bai_2,70.982440,72.804082,1.821642,NaN,NaN
9,cpg-n_0,19.711016,21.149214,1.438198,NaN,NaN


In [10]:
remade_weights

{'TTTTTT': 0.33771579948733754,
 'TTTTTC': 0.32202542990729677,
 'TTTTTA': 0.1185495022852104,
 'TTTTTG': 0.17971764061452333,
 'TTTCTT': 0.18682490621744058,
 'TTTCTC': 0.16794019732675733,
 'TTTCTA': 0.07794800808630908,
 'TTTCTG': 0.2873772919834115,
 'TTTATT': 0.3457112581775454,
 'TTTATC': 0.2739147801731458,
 'TTTATA': 0.1283502426150123,
 'TTTATG': 0.3889731232339006,
 'TTTGTT': 0.46320782905287866,
 'TTTGTC': 0.515278996273087,
 'TTTGTA': 0.30906415264784654,
 'TTTGTG': 1.0,
 'TTTTAT': 0.45838341253020215,
 'TTTTAC': 0.28109457103257,
 'TTTTAA': 0.1797353967106487,
 'TTTTAG': 0.10057795927572663,
 'TTTCAT': 0.3739819405120603,
 'TTTCAC': 0.31369494742948095,
 'TTTCAA': 0.17699755934347802,
 'TTTCAG': 0.3066091394232145,
 'TTTAAT': 0.49768849724532194,
 'TTTAAC': 0.4092424118334758,
 'TTTAAA': 0.43276676413485843,
 'TTTAAG': 0.5354774541669394,
 'TTTGAT': 1.0,
 'TTTGAC': 0.9939175688170555,
 'TTTGAA': 0.8677769323640374,
 'TTTGAG': 1.0,
 'TTTTCT': 0.5605097388044121,
 'TTTTCC': 

In [11]:
orig_weights

{'TTTTTT': 0.14153010255438922,
 'TTTTTC': 0.20508768693905877,
 'TTTTTA': 0.07025506134522225,
 'TTTTTG': 0.1836241280217444,
 'TTTCTT': 0.12295864397899496,
 'TTTCTC': 0.30842837751453855,
 'TTTCTA': 0.12459963155290636,
 'TTTCTG': 0.20639642532515187,
 'TTTATT': 0.43274498921346305,
 'TTTATC': 0.24082606193892556,
 'TTTATA': 0.11269123567843446,
 'TTTATG': 0.230578789855782,
 'TTTGTT': 0.36570411060364455,
 'TTTGTC': 0.5640140964258903,
 'TTTGTA': 0.3340564182846646,
 'TTTGTG': 1.0,
 'TTTTAT': 0.527602986974566,
 'TTTTAC': 0.41296852878950346,
 'TTTTAA': 0.31099687739350734,
 'TTTTAG': 0.36059624108878807,
 'TTTCAT': 0.28236016519909607,
 'TTTCAC': 0.26449684407387203,
 'TTTCAA': 0.09727966223525385,
 'TTTCAG': 0.3045182476831808,
 'TTTAAT': 0.43798565801349804,
 'TTTAAC': 0.31351127694232994,
 'TTTAAA': 0.5171822518864788,
 'TTTAAG': 0.3741723142667354,
 'TTTGAT': 1.0,
 'TTTGAC': 0.6959354306910741,
 'TTTGAA': 1.0,
 'TTTGAG': 0.6934061169424125,
 'TTTTCT': 0.33703712371546535,
 'TT